In [2]:
# just for checking the GPU name
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7d54c5bc-4ef0-48e7-68cd-8e2df4929681)


In [3]:
import os
import pandas as pd
from imageio import imread
import math
import numpy as np
import cv2
import keras
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam
from keras.models import Sequential, Model

In [4]:
def load_data(input_size=(64,64), data_path='GTSRB/Final_Training/Images'):

    pixels = []
    labels = []
    # loop over folders in images
    for dir in os.listdir(data_path):
        if dir == '.DS_Store':
            continue

        # read csv files for getting images data
        class_dir = os.path.join(data_path, dir)
        info_file = pd.read_csv(os.path.join(class_dir, 'GT-' + dir + '.csv'), sep=';')

        for row in info_file.iterrows():
            # read images
            pixel = imread(os.path.join(class_dir, row[1].Filename))
            # crop images by the ROI columns in csv file
            pixel = pixel[row[1]['Roi.X1']:row[1]['Roi.X2'], row[1]['Roi.Y1']:row[1]['Roi.Y2'], :]
            # resize to the standard size
            img = cv2.resize(pixel, input_size)

            # append image to data list
            pixels.append(img)

            # append label to list
            labels.append(row[1].ClassId)

    return pixels, labels

In [5]:
def split_train_val_test_data(pixels, labels):

    # standardize the data
    pixels = np.array(pixels)
    # this will convert the int list to one hot matrix
    labels = keras.utils.to_categorical(labels)

    # randomize data
    randomize = np.arange(len(pixels))
    np.random.shuffle(randomize)
    X = pixels[randomize]
    print("X=", X.shape)
    y = labels[randomize]

    # we will divide data into 60:20:20 portion
    # 60% is for training
    # 20% for validation
    # 20% for testing
    train_size = int(X.shape[0] * 0.6)
    X_train, X_val = X[:train_size], X[train_size:]
    y_train, y_val = y[:train_size], y[train_size:]

    val_size = int(X_val.shape[0] * 0.5)
    X_val, X_test = X_val[:val_size], X_val[val_size:]
    y_val, y_test = y_val[:val_size], y_val[val_size:]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [6]:
# building a model using VGG16 net (transfer learning)

def get_vgg16_model(input_shape=(64,64,3), output_size=43):
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

    # frozen the layers due to very carefully trained already
    for layer in model_vgg16_conv.layers:
        layer.trainable = False

    # build model
    input = Input(shape=input_shape, name='image_input')
    output_vgg16_conv = model_vgg16_conv(input)

    # add fully connected and dropout
    X = Flatten(name='flatten')(output_vgg16_conv)
    X = Dense(4096, activation='relu', name='fc1')(X)
    X = Dropout(0.5)(X)
    X = Dense(4096, activation='relu', name='fc2')(X)
    X = Dropout(0.5)(X)
    X = Dense(output_size, activation='softmax', name='predictions')(X)

    # compile
    model = Model(inputs=input, outputs=X)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [7]:
# buiding another model using a custom net

def get_conv2d_model(input_shape=(64,64,3), filter_size = (3,3), pool_size = (2, 2), output_size = 43):
    model = Sequential([
        Conv2D(16, filter_size, activation='relu', input_shape=input_shape, padding='same'),
        BatchNormalization(),
        Conv2D(16, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Dropout(0.2),
        Conv2D(32, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(32, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Dropout(0.2),
        Conv2D(64, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, filter_size, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=pool_size),
        Dropout(0.2),
        Flatten(),
        Dense(2048, activation='relu'),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(output_size, activation='softmax')
    ])
 
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])
    return model

In [8]:
# loading data from Google Drive
!unzip /content/drive/MyDrive/GTSRB.zip
data_path = '/content/GTSRB/Final_Training/Images'
pixels, labels = load_data(data_path=data_path)

Streaming output truncated to the last 5000 lines.
  inflating: GTSRB/Final_Training/Images/00010/00056_00024.ppm  
  inflating: __MACOSX/GTSRB/Final_Training/Images/00010/._00056_00024.ppm  
  inflating: GTSRB/Final_Training/Images/00010/00042_00024.ppm  
  inflating: __MACOSX/GTSRB/Final_Training/Images/00010/._00042_00024.ppm  
  inflating: GTSRB/Final_Training/Images/00010/00053_00003.ppm  
  inflating: __MACOSX/GTSRB/Final_Training/Images/00010/._00053_00003.ppm  
  inflating: GTSRB/Final_Training/Images/00010/00053_00017.ppm  
  inflating: __MACOSX/GTSRB/Final_Training/Images/00010/._00053_00017.ppm  
  inflating: GTSRB/Final_Training/Images/00010/00005_00029.ppm  
  inflating: __MACOSX/GTSRB/Final_Training/Images/00010/._00005_00029.ppm  
  inflating: GTSRB/Final_Training/Images/00010/00042_00018.ppm  
  inflating: __MACOSX/GTSRB/Final_Training/Images/00010/._00042_00018.ppm  
  inflating: GTSRB/Final_Training/Images/00010/00005_00001.ppm  
  inflating: __MACOSX/GTSRB/Final_Trai

In [9]:
X_train, y_train, X_val, y_val, X_test, y_test = split_train_val_test_data(pixels, labels)

X= (39209, 64, 64, 3)


In [10]:
vgg16_model = get_vgg16_model(input_shape=(64,64,3), output_size=43)
epochs = 50
batch_size = 64

vgg16_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                               validation_data=(X_val, y_val))

58892288/58889256 [==============================] - 0s 0us/step
Epoch 1/50
368/368 [==============================] - 21s 37ms/step - loss: 7.6845 - accuracy: 0.4832 - val_loss: 1.0853 - val_accuracy: 0.6929
Epoch 2/50
368/368 [==============================] - 13s 34ms/step - loss: 1.4870 - accuracy: 0.6674 - val_loss: 0.8866 - val_accuracy: 0.7737
Epoch 3/50
368/368 [==============================] - 13s 34ms/step - loss: 1.2147 - accuracy: 0.7281 - val_loss: 0.6970 - val_accuracy: 0.8095
Epoch 4/50
368/368 [==============================] - 13s 35ms/step - loss: 1.0375 - accuracy: 0.7582 - val_loss: 0.6556 - val_accuracy: 0.8264
Epoch 5/50
368/368 [==============================] - 13s 35ms/step - loss: 0.9957 - accuracy: 0.7725 - val_loss: 0.5898 - val_accuracy: 0.8415
Epoch 6/50
368/368 [==============================] - 13s 35ms/step - loss: 0.9677 - accuracy: 0.7853 - val_loss: 0.6178 - val_accuracy: 0.8387
Epoch 7/50
368/368 [==============================] - 13s 35ms/step - l

In [14]:
conv2d_model = get_conv2d_model(input_shape=(64,64,3), output_size=43)
epochs = 20
batch_size = 64

conv2d_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                               validation_data=(X_val, y_val))

Epoch 1/20
368/368 [==============================] - 7s 18ms/step - loss: 3.6430 - accuracy: 0.1094 - val_loss: 1.8051 - val_accuracy: 0.5180
Epoch 2/20
368/368 [==============================] - 6s 17ms/step - loss: 1.8838 - accuracy: 0.4617 - val_loss: 0.6864 - val_accuracy: 0.8123
Epoch 3/20
368/368 [==============================] - 6s 17ms/step - loss: 0.9539 - accuracy: 0.7117 - val_loss: 0.3044 - val_accuracy: 0.9112
Epoch 4/20
368/368 [==============================] - 6s 18ms/step - loss: 0.5508 - accuracy: 0.8333 - val_loss: 0.1988 - val_accuracy: 0.9411
Epoch 5/20
368/368 [==============================] - 6s 18ms/step - loss: 0.3580 - accuracy: 0.8925 - val_loss: 0.1141 - val_accuracy: 0.9662
Epoch 6/20
368/368 [==============================] - 6s 18ms/step - loss: 0.2285 - accuracy: 0.9288 - val_loss: 0.0932 - val_accuracy: 0.9717
Epoch 7/20
368/368 [==============================] - 6s 17ms/step - loss: 0.1662 - accuracy: 0.9504 - val_loss: 0.0628 - val_accuracy: 0.9832

In [12]:
vgg16_model.evaluate(X_test, y_test)

246/246 [==============================] - 4s 16ms/step - loss: 0.4034 - accuracy: 0.9232


[0.4034203588962555, 0.9232338666915894]

In [15]:
conv2d_model.evaluate(X_test, y_test)

246/246 [==============================] - 1s 4ms/step - loss: 0.0354 - accuracy: 0.9922


[0.035378456115722656, 0.9922213554382324]

Custom model seems to give us better result in this case, both for computing cost and evaluation